# BMW E30 M3 Service Manual - QLoRA Finetuning

This notebook finetunes **Llama-3.2-3B-Instruct** on BMW service manual data using QLoRA.

## Dataset Statistics
- Training: 1,185 examples (balanced across tasks)
- Validation: 158 examples
- Tasks: SPEC, PROCEDURE, EXPLANATION, WIRING, TROUBLESHOOTING

## Requirements
- GPU: T4 (Colab free), RTX 3060+, or better
- VRAM: ~8-10 GB
- Time: ~45-60 minutes on T4

## Cell 1: Setup & Authentication

In [ ]:
# Install required packages
!pip install -q accelerate peft bitsandbytes transformers trl datasets wandb

# Authenticate with HuggingFace
from huggingface_hub import notebook_login
notebook_login()  # Enter your HF token when prompted

## Cell 2: Mount Google Drive (for dataset upload)

**Before running**: Upload the following to Google Drive at `/content/drive/MyDrive/llm3/`:
- `config.yaml`
- `data/hf_train.jsonl`
- `data/hf_val.jsonl`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Verify files exist
import os
base_path = '/content/drive/MyDrive/llm3'
required_files = [
    f'{base_path}/config.yaml',
    f'{base_path}/data/hf_train.jsonl',
    f'{base_path}/data/hf_val.jsonl'
]

print("Checking required files...")
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ MISSING: {file}")

## Cell 3: Load Configuration and Datasets

In [ ]:
import yaml
import json
from datasets import Dataset

# Load config from Drive
config_path = '/content/drive/MyDrive/llm3/config.yaml'
with open(config_path) as f:
    config = yaml.safe_load(f)

print("📋 Configuration loaded:")
print(f"  Model: {config['huggingface']['model_name']}")
print(f"  Epochs: {config['training']['num_epochs']}")
print(f"  Batch size: {config['training']['per_device_train_batch_size']}")
print(f"  LoRA rank: {config['qlora']['r']}")

# Load datasets
def load_jsonl(path):
    data = []
    with open(path) as f:
        for line in f:
            data.append(json.loads(line))
    return Dataset.from_list(data)

train_dataset = load_jsonl('/content/drive/MyDrive/llm3/data/hf_train.jsonl')
val_dataset = load_jsonl('/content/drive/MyDrive/llm3/data/hf_val.jsonl')

print(f"\n✅ Loaded datasets - Train: {len(train_dataset)}, Val: {len(val_dataset)}")

# Show sample
print("\n📝 Sample training example:")
sample = train_dataset[0]
print(f"Task: {sample['meta']['task']}")
print(f"User: {sample['messages'][0]['content']}")
print(f"Assistant: {sample['messages'][1]['content']}")

## Cell 4: Load Llama 3.2 3B with QLoRA

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

model_name = config['huggingface']['model_name']
print(f"🔄 Loading model: {model_name}")

# 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# Load base model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True
)

# Set padding token (Llama doesn't have one by default)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✅ Tokenizer loaded - Vocab size: {len(tokenizer)}")

# Prepare model for k-bit training
model.config.use_cache = False  # Disable cache for training
model = prepare_model_for_kbit_training(model)

# Configure QLoRA
lora_config = LoraConfig(
    r=config['qlora']['r'],
    lora_alpha=config['qlora']['lora_alpha'],
    target_modules=config['qlora']['target_modules'],
    lora_dropout=config['qlora']['lora_dropout'],
    bias=config['qlora']['bias'],
    task_type=config['qlora']['task_type']
)

# Apply LoRA adapters
model = get_peft_model(model, lora_config)

# Print trainable parameters
print("\n📊 Trainable parameters:")
model.print_trainable_parameters()

## Cell 5: Prepare Dataset for Training

In [ ]:
# Format dataset for SFT (Supervised Fine-Tuning)
def format_chat(example):
    """Apply Llama 3.2 chat template"""
    text = tokenizer.apply_chat_template(
        example['messages'], 
        tokenize=False, 
        add_generation_prompt=False
    )
    return {"text": text}

# Apply formatting
print("🔄 Formatting datasets with chat template...")
train_dataset = train_dataset.map(format_chat, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(format_chat, remove_columns=val_dataset.column_names)

print(f"✅ Formatted datasets ready")
print(f"\n📝 Sample formatted text (first 500 chars):\n")
print(train_dataset[0]['text'][:500] + "...")

## Cell 6: Configure Training Arguments

In [ ]:
from transformers import TrainingArguments

output_dir = "./bmw_e30_qlora_results"

# Convert learning_rate to float if it's a string (YAML parsing issue)
lr = float(config['training']['learning_rate'])

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=config['training']['num_epochs'],
    per_device_train_batch_size=config['training']['per_device_train_batch_size'],
    per_device_eval_batch_size=config['training']['per_device_train_batch_size'],
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
    gradient_checkpointing=True,  # Saves memory
    learning_rate=lr,  # Use the converted float
    warmup_ratio=config['training']['warmup_ratio'],
    weight_decay=config['training']['weight_decay'],
    logging_steps=config['training']['logging_steps'],
    logging_dir=f"{output_dir}/logs",
    save_strategy=config['training']['save_strategy'],
    eval_strategy="epoch",
    eval_steps=None,
    save_total_limit=2,  # Only keep 2 best checkpoints
    fp16=True,  # Mixed precision training
    bf16=False,
    optim=config['training']['optim'],
    report_to="none",  # Change to "wandb" if you want tracking
    push_to_hub=False,
    max_grad_norm=0.3,  # Gradient clipping
    lr_scheduler_type="cosine"
)

effective_batch = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
total_steps = (len(train_dataset) // effective_batch) * training_args.num_train_epochs

print("✅ Training arguments configured")
print(f"📊 Effective batch size: {effective_batch}")
print(f"⏱️  Total training steps: ~{total_steps}")
print(f"🔥 Warmup steps: ~{int(total_steps * training_args.warmup_ratio)}")

## Cell 7: Initialize Trainer and Start Training

**Expected training time**:
- T4 (Colab free): ~45-60 minutes
- A100: ~15-20 minutes

In [ ]:
from trl import SFTTrainer

# Formatting function to extract text from dataset
def formatting_func(example):
    return example["text"]

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=training_args,
    formatting_func=formatting_func
)

print("🚀 Starting training...")
print(f"⏱️  Estimated time: 1-2 hours on T4, 30-45 min on A100\n")

# Train the model
trainer.train()

print("\n✅ Training complete!")

## Cell 8: Evaluate on Validation Set

In [ ]:
# Evaluate
print("📊 Evaluating on validation set...")
eval_results = trainer.evaluate()

print("\n📈 Evaluation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

## Cell 9: Save Model Locally

In [ ]:
# Save the fine-tuned model
save_dir = "./bmw_e30_m3_service_manual"

print(f"💾 Saving model to {save_dir}...")
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print("✅ Model saved locally")

# Also save to Google Drive for persistence
import shutil
drive_save_dir = '/content/drive/MyDrive/llm3/models/bmw_e30_m3_service_manual'
print(f"\n💾 Copying to Google Drive: {drive_save_dir}...")
shutil.copytree(save_dir, drive_save_dir, dirs_exist_ok=True)
print("✅ Model saved to Google Drive")

## Cell 10: Test Inference (Quick Validation)

In [ ]:
# Quick inference test
def test_model(prompt):
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the assistant's response
    response = response.split("assistant\n")[-1] if "assistant" in response else response
    return response

# Test cases
print("🧪 Testing model with sample queries:\n")

test_queries = [
    "[SPEC] What is the torque for cylinder head bolts?",
    "[PROCEDURE] How do you adjust valve clearance?",
    "[EXPLANATION] Explain the Motronic control unit operation"
]

for query in test_queries:
    print(f"❓ Query: {query}")
    print(f"💬 Response: {test_model(query)}\n")
    print("-" * 80 + "\n")

## Cell 11: Push to HuggingFace Hub (Optional)

**Note**: Change `hub_model_id` to your HuggingFace username before running!

In [ ]:
# Push to HuggingFace Hub
hub_model_id = "your-username/bmw-e30-m3-service-manual"  # ⚠️ Change to your username!

print(f"🚀 Pushing model to HuggingFace Hub: {hub_model_id}")
print("⏱️  This may take a few minutes...\n")

model.push_to_hub(hub_model_id, use_auth_token=True)
tokenizer.push_to_hub(hub_model_id, use_auth_token=True)

print(f"✅ Model successfully pushed!")
print(f"🔗 View at: https://huggingface.co/{hub_model_id}")

## Cell 12: Load from Hub (Test Deployment)

In [ ]:
# Test loading from HuggingFace Hub
from peft import PeftModel, PeftConfig

print(f"🔄 Loading model from Hub: {hub_model_id}")

# Load config
peft_config = PeftConfig.from_pretrained(hub_model_id)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# Load adapter
model_from_hub = PeftModel.from_pretrained(base_model, hub_model_id)

print("✅ Model loaded from Hub successfully!")

# Quick test
test_prompt = "[SPEC] What is the engine displacement?"
print(f"\n🧪 Test query: {test_prompt}")
messages = [{"role": "user", "content": test_prompt}]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(model_from_hub.device)
outputs = model_from_hub.generate(**inputs, max_new_tokens=50, temperature=0.7, do_sample=True)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"💬 Response: {response.split('assistant')[-1].strip() if 'assistant' in response else response}")

## Next Steps

1. **Evaluate thoroughly**: Test on diverse queries from the validation set
2. **Monitor for overfitting**: Check if train/eval loss diverged
3. **Adjust hyperparameters** if needed:
   - Increase LoRA rank (16 → 32) if underfitting
   - Increase dropout (0.05 → 0.1) if overfitting
   - Train for more epochs if loss still decreasing
4. **Deploy**: Use the model from HuggingFace Hub for inference
5. **Collect feedback**: Test with real BMW technicians if possible

## Resources

- [QLoRA Paper](https://arxiv.org/abs/2305.14314)
- [Llama 3.2 Model Card](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)
- [PEFT Documentation](https://huggingface.co/docs/peft)
- [TRL Documentation](https://huggingface.co/docs/trl)